# SARSA
---
> Temporal Differential Prediction + Epsiolon - Greedy update

* D.P 는 환경에 대한 확실한 확률정보를 알고있는 모델을 푸는 과정
* 환경에 불확실성이 포함된 모델을 푸는것이 RL 알고리즘
* 주로 sampling 기법으로 불확실성을 예측한다.
    * 그래서 업데이트 식을 보면 평균을 구하지 않고 샘플 하나에 대해서만 계산한다
* SARSA 알고리즘은 Temporal Difference 예측 샘플링 기법을 사용함.
* 한꺼번에 모두 업데이트하는 D.P 와 다르게 한 step 씩 계산한다.
* Bellman Equation 을 변형하여 sampling 으로 얻은 data 를 적용한다.
* Policy Iteration + Greedy action select --> On - Policy
* bootstrap 알고리즘
    * M.C 와 다르게 업데이트 식에 이전 q (추정) 가 들어감 -> 추정이 쌓임
    * M.C 는 실 관측 reward-to-go 값으로 업데이트, sarsa 는 현재 reward 만 신호로 사용
    * bootstrap 이 self-consistency equation 으로 볼 수도 있음
* 역시 model free 하기 위해 V 대신 Q 를 사용
    * V, Q 를 학습하는 모든 RL 알고리즘에서 명심할 것 - 가치라는 것 자체에 추정의 의미가 들어감.
    * DP 는 추정이 아니라 true 인데, 샘플링한다는 점에서 추정이 됨.
    * 근데 만약 신경망 학습을 한다면 추정이 한 단계 더 (근사) 들어가 더욱 불안정해질 가능성이 있음.
    * deep sarsa, dqn 등이 해당됨
* value based 알고리즘이므로 action 을 만드는 policy 는 implicit
* 하지만 action 을 만드는 policy 와 상호작용하는 policy 가 같아서 문제가 생김 (on-policy)
    * 식의 미묘한 차이를 알아차리기 어려움
    * 같은 규칙 (정책) 에 의해 샘플링된 action 이 Q 를 업데이트하는 데 동일한 규칙을 사용한다

In [1]:
import gym
import numpy as np
import random
from gym.envs.registration import register

In [2]:
'''
환경셋팅 한 후에 환경을 추가등록한다.
'''

register(
    id='FrozenLake-v1',
    entry_point="gym.envs.toy_text:FrozenLakeEnv",
    kwargs={'map_name':'4x4','is_slippery':False})

In [3]:
'''
환경 생성
'''
env = gym.make('FrozenLake-v1')

In [183]:
q_table = np.zeros([env.action_space.n, env.observation_space.n], dtype = np.float16)
gamma = .9
epsilon = 1
episode = 0
max_episode = 1000

state = env.reset()
action = env.action_space.sample()
step = 0

while(episode < max_episode):
    
    #env.render()
    step += 1
    state_next, reward, done, _ = env.step(action)
    #reward -= step*.01
        
    if(random.random() > epsilon):
        action_next = np.argmax(q_table[ : , state_next])
    else:
        action_next = env.action_space.sample()
    
    # SARSA 업데이트
    q_table[action, state] += gamma * (reward + q_table[action_next, state_next] - q_table[action, state])
    
    state_old = state
    action_old = action
    
    state = state_next
    action = action_next
    
    if(done):
        if(reward):
            if(epsilon > .1):
                epsilon = 1 / (episode/(max_episode/10) + 1)
            else:
                epsilon = .1
        else:
            q_table[action_old, state_old] += gamma * (-1 + q_table[action, state] - q_table[action_old, state_old])
        
        step = 0
        episode += 1
        env.reset()
env.close()

In [184]:
q_table

array([[-5.293 , -5.27  , -4.883 , -2.89  , -4.203 , -5.04  , -5.84  ,
        -4.87  , -5.832 , -5.2   , -5.34  , -4.418 , -3.5   , -4.926 ,
        -4.266 , -1.952 ],
       [-1.757 , -4.965 , -4.613 , -5.707 , -2.773 , -5.504 , -0.5864,
        -2.182 , -5.875 , -0.916 , -0.771 , -4.723 , -3.158 , -6.043 ,
        -3.59  , -5.55  ],
       [-5.17  , -5.492 , -4.688 , -5.598 , -6.21  , -5.293 , -5.703 ,
        -5.957 , -1.913 , -4.723 , -3.963 , -5.426 , -5.074 , -1.053 ,
        -0.965 , -4.555 ],
       [-4.07  , -5.285 , -4.63  , -5.18  , -6.184 , -4.645 , -5.246 ,
        -5.082 , -5.332 , -5.55  , -4.047 , -1.748 , -5.062 , -5.19  ,
        -4.84  , -5.09  ]], dtype=float16)

In [185]:
s = env.reset()

step = 1
while(True):
    #env.render()
    a = np.argmax(q_table[ : , s])
    s,r,d,_ = env.step(a)
    if(d):
        env.render()
        break
env.close()


SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
